In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.ocrdata import load_dataset
# from dataset.mnist import load_mnist
# from deep_convnet import DeepConvNet
from common.trainer import Trainer

(x_train, t_train), (x_test, t_test) = load_dataset()

network = DeepConvNet(output_size=26)  
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=20, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr':0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보관
network.save_params("deep_convnet_params.pkl")
print("Saved Network Parameters!")

NameError: name 'DeepConvNet' is not defined

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
# from dataset.mnist import load_mnist
from dataset.ocrdata import load_dataset
from net.simple_convnet import SimpleConvNet
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_dataset()

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=26, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:4.27087403534
=== epoch:1, train acc:0.047, test acc:0.052 ===
train loss:4.12515665013
train loss:4.11468749083
train loss:4.00323870758
train loss:4.05159357275
train loss:3.884894654
train loss:3.77638894927
train loss:3.59038852345
train loss:3.20060396763
train loss:2.85972329822
train loss:2.65060468079
train loss:2.60300589698
train loss:2.30690821321
train loss:2.01804076383
train loss:1.87756536355
train loss:1.56288222882
train loss:1.92800730064
train loss:1.65245039194
train loss:1.41047535731
train loss:1.4893128952
train loss:1.41095875647
train loss:1.25763292373
train loss:1.13674035441
train loss:1.15550475598
train loss:1.10898893126
train loss:1.40022574692
train loss:0.981241687333
train loss:1.31549760965
train loss:1.05723399223
train loss:0.967939320825
train loss:1.27675976801
train loss:0.881094427741
train loss:0.843580267942
train loss:1.05675880235
train loss:0.668098116263
train loss:0.82779585116
train loss:0.885728632689
train loss:0.8853431680

In [1]:
# coding: utf-8
import numpy as np
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.layers import *

class DeepConvNet:
    def __init__(self, input_dim=(1,28,28), 
                conv_params1={'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                conv_params2={'filter_num':16, 'filter_size':3, 'pad':1, 'stride':1},
                conv_params3={'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                conv_params4={'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                conv_params5={'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                conv_params6={'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                hidden_size=100, output_size=26):
        pre_node_num = np.ones(8, dtype=np.uint16)
        pre_channel_num = input_dim[0]
        for idx, conv_params in enumerate([conv_params1, conv_params2, conv_params3, 
                                           conv_params4, conv_params5, conv_params6]):
            pre_node_num[idx] = pre_channel_num * conv_params['filter_size'] ** 2
            pre_channel_num = conv_params['filter_num']
        pre_node_num[6] = conv_params6['filter_num'] * 4 * 4
        pre_node_num[7] = hidden_size
        print(pre_node_num)
        weight_init_std = np.sqrt(2/pre_node_num)
        weight_init_std = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])
        
        self.params = {}
        channel_num = input_dim[0]
        for idx, conv_params in enumerate([conv_params1, conv_params2, conv_params3, 
                                           conv_params4, conv_params5, conv_params6]):
            self.params['W' + str(idx + 1)] = weight_init_std[idx] * np.random.randn(
                conv_params['filter_num'], channel_num, conv_params['filter_size'], conv_params['filter_size'])
            self.params['b' + str(idx + 1)] = np.zeros(conv_params['filter_num'])
            channel_num = conv_params['filter_num']
        self.params['W7'] = weight_init_std[6] * np.random.randn(pre_node_num[6], hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = weight_init_std[7] * np.random.randn(pre_node_num[7], output_size)
        self.params['b8'] = np.zeros(output_size)
        
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_params1['stride'], conv_params1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_params2['stride'], conv_params2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_params3['stride'], conv_params3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_params4['stride'], conv_params4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_params5['stride'], conv_params5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_params6['stride'], conv_params6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(DropOut(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(DropOut(0.5))

        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, DropOut):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)

        return x
    
    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)
    
    def accuracy(self, x, t, batch_size=100):
        acc = 0.0
        
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        acc = np.sum(y==t)/x.shape[0]
        return acc
    
    def gradient(self, x, t):
        self.loss(x, t)
        grads = {}
        dout = 1
        dout = self.last_layer.backward(dout)
        
        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)
            
        for i, layer_idx in enumerate([0, 2, 5, 7, 10, 12, 15, 18]):
            grads['W' + str(i + 1)] = self.layers[layer_idx].dW
            grads['b' + str(i + 1)] = self.layers[layer_idx].db
            
        return grads
    
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]
            
            